In [14]:
from jenga.tasks.openml import OpenMLBinaryClassificationTask, OpenMLMultiClassClassificationTask
from jenga.corruptions.generic import MissingValues
    
import pandas as pd
import numpy as np

from data_imputation_paper.imputation.simple import ModeImputer
from data_imputation_paper.imputation.ml import KNNImputer, ForestImputer
from data_imputation_paper.imputation.dl import AutoKerasImputer
from data_imputation_paper.imputation.generative import GAINImputer
from data_imputation_paper.evaluation import Evaluator

ImportError: cannot import name 'OpenMLBinaryClassificationTask' from 'jenga.tasks.openml' (/Users/sebastian/code/data_imputation/data-imputation-paper/src/jenga/src/jenga/tasks/openml.py)

## Make Deterministic

In [15]:
seed = 42

## Create example tasks

In [16]:
task = OpenMLMultiClassClassificationTask(openml_id=4552, seed=seed)

NameError: name 'OpenMLMultiClassClassificationTask' is not defined

## Insert missing values using jenga

In [4]:
missing_values = [
    MissingValues(column='V2', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V4', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V15', fraction=0.5, na_value=np.nan, missingness='MCAR')
]

## Mode Imputation

In [5]:
arguments = {
    "seed": seed
}

%time Evaluator(task, missing_values, ModeImputer, arguments).evaluate(10).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     29.703142    29.392351
MSE   1390.874970  1378.768030
RMSE    37.292379    37.124744

Target Column: V4
                train      test
F1_micro     0.832215  0.821201
F1_macro     0.454211  0.450896
F1_weighted  0.756009  0.740611

Target Column: V15
                train      test
F1_micro     0.117079  0.118551
F1_macro     0.013100  0.013244
F1_weighted  0.024558  0.025202

CPU times: user 2.26 s, sys: 20.9 ms, total: 2.29 s
Wall time: 2.28 s


## KNN imputation

In [6]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_neighbors": [3, 5]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_neighbors": [3, 5]
    }
}

%time Evaluator(task, missing_values, KNNImputer, arguments).evaluate(5).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     30.943813    30.024876
MSE   1524.363449  1413.293710
RMSE    39.042549    37.586951

Target Column: V4
                train      test
F1_micro     0.939541  0.931449
F1_macro     0.886165  0.874654
F1_weighted  0.938422  0.929498

Target Column: V15
                train      test
F1_micro     0.523742  0.503887
F1_macro     0.426132  0.398130
F1_weighted  0.516240  0.493827

CPU times: user 12.5 s, sys: 2.77 s, total: 15.3 s
Wall time: 15.7 s


## Forest imputation

In [7]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_estimators": [50, 100]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_estimators": [50, 100]
    }
}

%time Evaluator(task, missing_values, ForestImputer, arguments).evaluate(5).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     27.956981    27.123680
MSE   1259.440153  1172.663605
RMSE    35.486581    34.242265

Target Column: V4
                train      test
F1_micro     0.951545  0.948057
F1_macro     0.909422  0.904348
F1_weighted  0.950198  0.947083

Target Column: V15
                train      test
F1_micro     0.564784  0.546996
F1_macro     0.469239  0.454020
F1_weighted  0.560843  0.541598

CPU times: user 44.2 s, sys: 1.2 s, total: 45.4 s
Wall time: 38.7 s


## AutoKeras imputation

In [ ]:
arguments = {
    "seed": seed,
    'max_trials': 2,
    'tuner': 'greedy',
    'validation_split': 0.1,
    'epochs': 2
}

%time Evaluator(task, missing_values, AutoKerasImputer, arguments).evaluate(5).report()

## GAIN imputation

In [8]:
arguments = {
    "num_data_columns": task.train_data.shape[1],
    "seed": seed,
    "hyperparameter_grid": {
        "gain": {
            "alpha": [80, 120],
            "hint_rate": [0.5, 0.9],
            "noise": [0.001, 0.1]
        },
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        }
    }
}

%time Evaluator(task, missing_values, GAINImputer, arguments).evaluate(2).report()

[2021-02-22 14:59:50,341 - base_layer.py:1814 - WARNING - _warn_about_input_casting]: Layer concatenate is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[2021-02-22 14:59:50,341 - base_layer.py:1814 - WARNING - _warn_about_input_casting]: Layer concatenate is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[W 2021-02-22 14:59:50,358] Trial 0 failed because of the following error: TypeError('in converted code:\n\n    /Users/aallhorn/dev/data-imputation-paper/src/data_imputation_paper/imputation/generative.py:147 train_step  *\n        generater_loss, discriminator_loss = self.gain([X, M, H])\n    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__\n        outputs = call_fn(cast_inputs, *args, **kwargs)\n    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:708 call\n        convert_kwargs_to_constants=base_layer_utils.call_context().saving)\n    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:860 _run_internal_graph\n        output_tensors = layer(computed_tensors, **kwargs)\n    /usr/local/Caskroom/miniconda/base/e

TypeError: in converted code:

    /Users/aallhorn/dev/data-imputation-paper/src/data_imputation_paper/imputation/generative.py:147 train_step  *
        generater_loss, discriminator_loss = self.gain([X, M, H])
    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:708 call
        convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:860 _run_internal_graph
        output_tensors = layer(computed_tensors, **kwargs)
    /usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.7/site-packages/tensorflow_core/python/keras/en